In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 0. 환경변수 및 패키지

In [21]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_ollama import ChatOllama
import os
import re
import ollama

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_upstage import ChatUpstage

from typing import List, Optional, Dict, Any
from __future__ import annotations
from collections import defaultdict

load_dotenv()

True

In [10]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_LLM_MODEL = 'gpt-4o-mini'
OPENAI_EMBEDDIGN_MODEL = 'text-embedding-3-large' 
UPSTAGE_EMBEDDIGN_MODEL = 'solar-embedding-1-large-passage'
OLLAMA_EMBEDDING_MODEL = 'bge-m3'

PINECONE_INDEX_NAME_LAW = 'law-index-final'
PINECONE_INDEX_NAME_RULE = 'rule-index-final'
# PINECONE_INDEX_NAME_CASE = 'case-index-final'

COHERE_API_KEY = os.getenv('COHERE_API_KEY')

# 1. 함수

In [23]:
def get_law_category():
    """
    주택임대차 RAG 시스템을 위한 카테고리-키워드 가중치 사전
    Returns:
        dict: {카테고리명: {키워드: 가중치}}
    """
    return {
        # 1. 보증금 보호 (가장 치명적인 리스크)
        '보증금_대항력': {
            '보증금': 3, '전세금': 2, '보증금반환': 3, '반환청구': 2,
            '대항력': 3, '전입신고': 2, '주민등록': 2, '점유': 2, '인도': 2,
            '우선변제권': 3, '최우선변제': 3, '소액임차인': 3, '배당': 2, '경매': 2, '공매': 2,
            '확정일자': 3, '확정일자부': 2,'임차권등기': 3, '임차권등기명령': 3,
            '반환보증': 2, '보증보험': 2, 'HUG': 1, '보증보험': 2,
            '대항요건': 3,'제삼자': 2,'효력': 1,'효력발생': 2, '입주': 1,
            '거주': 1,'주택인도': 2, '임대차계약증서': 2,'환가대금': 2,
            '후순위권리자': 2,'변제': 1,'배당요구': 2,'임차권등기명령': 3,'임대차등기': 2,
            '지방법원': 2,'등기촉탁': 1,'등기신청': 1,
        },

        # 2. 계약 갱신 및 거절 (가장 빈번한 분쟁)
        '계약갱신': {
            '계약갱신': 3, '갱신요구': 3, '갱신요구권': 3, '갱신거절': 3, '묵시적갱신': 3,
            '계약연장': 2, '재계약': 2, '존속기간': 2, '2년': 1,
            '실거주': 3, '거절사유': 2, '손해배상': 2,
            '연체': 2, '2기': 2, '의무위반': 2,
            '재계약': 2, '갱신청구': 2, '거절사유': 2, '복비': 1, '계약갱신청구권': 3,
            '갱신거절통지': 3,'계약조건변경': 2, '계약만료': 2,'직계존속': 2,
            '직계비속': 2,'직계존비속': 2, '중개보수': 1,
        },

        # 3. 계약 해지 및 종료 (탈출 전략)
        '계약해지': {
            '계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2,
            '계약종료': 2, '퇴거': 2, '이사': 1, '명도': 2,
            '합의해지': 2, '통지': 1, '3개월': 2, '즉시해지': 3
        },

        # 4. 차임(월세) 및 보증금 증감
        '임대료_증감': {
            '월세전환': 3, '반전세': 2, '차임': 3, '월세': 2, '증액': 3, 
            '5퍼센트': 3, '5%': 3, '20분의 1': 2, '상한': 2, '감액': 2, '인상': 2, '1년': 2, '1년 이내': 2,
            '전환율': 2, '월차임': 2, '경제사정': 1, '부담': 1, '전환되는 금액': 2,
            '초과차임': 2, '반환': 2, '증액청구': 3,'증액비율': 2, '차임액': 2,
            '보증금증액': 2, '차임연체': 2, '2기차임': 2,'연체': 1,
        },

        # 5. 유지보수 및 원상회복 (민법 영역)
        '수선_원상회복': {
            '수선': 3, '수리': 3, '원상회복': 3, '파손': 2, '훼손': 2, '유지': 2, '하자': 2,
            '누수': 2, '곰팡이': 2, '보일러': 2, '필요비': 3, '유익비': 3,
            '비용상환': 2, '보존행위': 2, '관리비': 1
        },

        # 6. 전세사기 및 권리 리스크 (독소조항 탐지용)
        '권리_리스크': {
            '전세사기': 3, '깡통전세': 3, '신탁': 3, '근저당': 3, '저당권': 3, '신탁부동산': 3,
            '선순위': 3, '가압류': 3, '압류': 3, '국세': 2, '지방세': 2,
            '체납': 3, '납세증명': 2, '위반건축물': 2, '불법건축물': 2,
            '특약': 2, '독소조항': 2, '강행규정': 3, '효력이 없다': 3, 
            '무효': 3, '불리한 약정': 3, '편면적 강행규정': 2, '등기부등본': 2,
            '말소기준권리': 3, '채권최고액': 2, '가등기': 2, '전세권': 2, '전세권등기': 2,
            '소유권이전등기': 2,'말소': 2, '배당요구': 2,'건축물대장': 2, '보존등기': 2, '이행강제금': 1,
        },

        # 7. 행정 절차 (Rules Index 연결)
        '행정절차': {
            '확정일자부여': 3, '동주민센터': 2, '등기소': 2, '인터넷등기소': 3,
            '수수료': 1, '열람': 2, '제공요청': 2, '이해관계인': 2,
            '임대차계약증서': 2, '전자계약': 2, '신분증': 1, '등기부등본': 3,
            '건축물대장': 3,'중개대상물확인설명서': 2,'공인중개사': 1,'중개수수료': 2, '중개보수': 2,
        },

        # 8. 소송 및 분쟁 해결 (최후 수단)
        '분쟁해결': {
            '분쟁조정': 3, '조정위원회': 2, '지급명령': 3, '소송': 3, '조정': 3, '조정부': 2,
            '판결': 2, '집행권원': 2, '경매': 3, '공매': 2,
            '내용증명': 2, '손해배상': 3, '지연이자': 2, '조정신청': 3, '신청': 2, '각하': 2,
            '조정절차': 3, '송달': 2, '서면': 2, '구두': 2,
            '처리기간': 2, '60일': 2, '30일': 1,
            '심사관': 1, '조사관': 1, '사무국': 1,
            '조정신청': 3,'조정안': 2,'조정서': 2, '조정성립': 2,'인도명령': 2,'명도소송': 2,
            
            # 소송/집행 트리거
            '지급명령': 3, '소송': 3, '판결': 2, '집행권원': 2,
            '경매': 3, '공매': 2, '내용증명': 2, '손해배상': 3, '지연이자': 2,

            # 시행령(원문 표기): 조정 대상 범위 + 신청 절차 상세
            '제22조(조정위원회의 심의ㆍ조정 사항)': 3,
            '제29조(조정부의 구성 및 운영)': 1,
            '제30조(조정의 신청)': 2,
            '제31조(조정신청인에게 안내하여야 할 사항)': 1,
            '제32조(조정서류의 송달 등)': 1,
        },
        
        # 9. 임차권 승계 및 가족 
        '임차권_승계': {
            '임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '상속인': 3,
            '사실혼': 3, '배우자': 2, '가정공동생활': 3, '2촌': 2, 
            '공동상속': 2, '반환청구권': 2
        },
        
        # 10. 전대/양도
        '전대_양도': {
            '전대': 3, '전차': 2, '전대차': 2, '임대인의 동의': 3,
            '양도': 2, '임차권 양도': 2
        },

        # 0. 적용 범위/예외(필요 시 사용)
        '적용범위_예외': {
            '적용범위': 3, '주거용': 2, '주택 일부': 2, '주거 외 목적': 2,
            '일시사용': 3, '일시사용하기 위한 임대차': 3,
            '미등기': 3, '미등기 전세': 3, '전세계약': 2
        },
    }


def categorize_content(content, top_k=None):
    '''
    내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters : 
        - content : 분류할 텍스트 내용
        - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
        
    Returns :
        - 카테고리 리스트(점수 높은 순)
    '''
    category_keywords = get_law_category()
    category_scores = {}
    
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        score = 0
        for keyword, weight in weighted_keywords.items():
            count = content.count(keyword) # content에 keyword가 몇번 나오는지
            score += count * weight
        if score > 0 :
            category_scores[category] = score
    
    # 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x : x[1], reverse=True)
    all_categories = [category[0] for category in sorted_categories]
    
    # 매칭되는 카테고리가 없으면 '기타' 반환
    if not all_categories:
        all_categories = ['기타']
        
    # top_k가 지정되면 상위 top_k개만, 아니면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    return all_categories


# =====================
# 참조조항을 추출하는 함수
# =====================

_ARTICLE_RE = re.compile(r"(?:제\s*)?(\d+)\s*조(?:\s*의\s*(\d+))?")

def _law_name_from_src_title(src_title: str) -> str:
    if not src_title:
        return "알수없음"
    s = str(src_title).strip()
    if not s:
        return "알수없음"
    # 첫 '(' 이전을 법령명으로 사용
    name = s.split("(", 1)[0].strip()
    return name if name else "알수없음"

def _split_article_field(article_field) -> list[str]:
    if not article_field or article_field == "조항없음":
        return []
    s = str(article_field)
    s = s.replace("「", "").replace("」", "").strip()
    if not s:
        return []
    # ",", ";", 줄바꿈, "·" 등 다양한 구분자 대응
    tokens = re.split(r"[,\n;·]+", s)
    return [t.strip() for t in tokens if t and t.strip()]

def _parse_article_token(token: str):
    if not token:
        return None
    token = re.sub(r"\s+", "", token)
    m = _ARTICLE_RE.search(token)
    if not m:
        return None
    main = int(m.group(1))
    sub = int(m.group(2)) if m.group(2) else 0
    return (main, sub)

def _format_article_key(key):
    main, sub = key
    return f"제{main}조" if sub == 0 else f"제{main}조의{sub}"

def extract_articles_from_docs(documents):
    """
    검색된 문서들(documents)에서 article 정보를 추출하여
    '법령명: 제x조,제y조 | 법령명2: ...' 형태로 반환
    """
    # law_name -> set( (main,sub) )
    buckets = defaultdict(set)
    # law_name -> best priority(작을수록 우선)
    law_priority = {}

    for doc in documents:
        meta = doc.metadata or {}

        law = _law_name_from_src_title(meta.get("src_title") or meta.get("law_name") or meta.get("source"))
        pr = meta.get("priority", None)
        try:
            pr_int = int(pr) if pr is not None else None
        except Exception:
            pr_int = None

        if pr_int is not None:
            prev = law_priority.get(law)
            if prev is None or pr_int < prev:
                law_priority[law] = pr_int

        for tok in _split_article_field(meta.get("article", "조항없음")):
            key = _parse_article_token(tok)
            if key is not None:
                buckets[law].add(key)

    if not buckets:
        return "(참조 조항 없음)"

    # 법령 블록 정렬: priority(있으면) -> 법령명
    def law_sort_key(law_name: str):
        p = law_priority.get(law_name)
        return (p if p is not None else 9999, law_name)

    parts = []
    for law in sorted(buckets.keys(), key=law_sort_key):
        keys = sorted(buckets[law])
        if not keys:
            continue
        parts.append(f"{law} " + ",".join(_format_article_key(k) for k in keys))

    return " | ".join(parts) if parts else "(참조 조항 없음)"



# ====================================
# 검색된 document를 텍스트로 변환하는 함수
# ====================================

def format_documents(documents):
    return '\n\n---\n\n'.join([doc.page_content for doc in documents])


# ===========================
# title+text 기반의 rerank함수
# ===========================

import cohere
import numpy as np

def rerank_by_title(query: str, documents: list, top_k: int = 4):
    '''
    cohere의 rerank API를 사용한 문서 재정렬
    '''
    # cohere client 초기화
    co = cohere.Client(api_key=COHERE_API_KEY)
    
    # 0건이면 rerank 호출하지 말고 그대로 반환
    if not documents:
        return []

    # top_k도 0이면 그대로
    if not top_k or top_k <= 0:
        return documents

    docs_text = [
        (doc.metadata.get("src_title", "") or "") + " " + (doc.page_content or "")
        for doc in documents
    ]

    # documents가 비어있지 않아도, docs_text가 전부 빈문자면 Cohere가 싫어할 수 있어서 방어
    if not any(t.strip() for t in docs_text):
        return documents[:top_k]

    results = co.rerank(
        model="rerank-multilingual-v3.0",
        query=query,
        documents=docs_text,
        top_n=min(top_k, len(docs_text)),
    )

    idxs = [r.index for r in results.results]
    return [documents[i] for i in idxs]


# ==========================================
# 검색된 document를 LLM용 context로 변환 (개선)
# ==========================================

def format_documents_with_meta(documents, max_len: int = 500):
    """
    프롬프트.txt 구조 반영:
    - 법령명 / 조문 / priority 명시
    - LLM이 법적 근거를 명확히 인식하도록 설계
    """
    context_parts = []

    for i, doc in enumerate(documents, 1):
        meta = doc.metadata or {}

        law_name = meta.get("law_name") or meta.get("src_title") or meta.get("source", "알 수 없는 법령")
        article = meta.get("article", "")
        priority = meta.get("priority", "?")

        content = doc.page_content.strip()
        if len(content) > max_len:
            content = content[:max_len].rstrip() + "…"

        context_parts.append(
            f"""[문서 {i}]
- 법령명: {law_name}
- 조문: {article}
- 우선순위(priority): {priority}
- 내용:
{content}
"""
        )

    return "\n\n".join(context_parts)

## 우선순위

In [5]:
import re

def sanitize_for_upstage(text: str, max_len: int = 500) -> str:
    """Upstage embedding input 안전 정제: str 보장, 공백/개행 정리, 길이 제한"""
    if text is None:
        return ""
    if not isinstance(text, str):
        text = str(text)

    text = text.strip()

    # 이상한 프롬프트 잔재 제거(가끔 '질문 :' 같은 prefix가 섞임)
    text = re.sub(r"^\s*(질문\s*:|question\s*:|q\s*:)\s*", "", text, flags=re.IGNORECASE)

    # 개행/탭 과다를 단일 공백으로
    text = re.sub(r"\s+", " ", text).strip()

    # 너무 길면 자르기
    if len(text) > max_len:
        text = text[:max_len].strip()

    return text

def safe_embedding_query(normalized_query: str, original_query: str) -> str:
    q = sanitize_for_upstage(normalized_query)
    if not q:  # 빈 문자열이면 원문으로 폴백
        q = sanitize_for_upstage(original_query)
    return q


def retrieve_law_docs_priority_mix(
    law_db,
    embedding,
    query_for_embed: str,
    categories: Optional[List[str]] = None,
    k_total: int = 6,
    k_each: int = 8,
    # ✅ 우선순위 정책: 1(법) 먼저, 2(시행령) 최소 확보, 그 다음 4(민법) 등
    priority_order: List[int] = [1, 2, 4, 3, 5, 6, 7],
    # ✅ 최소 할당량(시행령 1개 확보)
    min_quota: Optional[Dict[int, int]] = None,
):
    if min_quota is None:
        min_quota = {1: max(1, k_total - 1), 2: 1}  # 기본: 1에서 k_total-1, 2에서 1

    qvec = embedding.embed_query(query_for_embed)
    picked: List[Document] = []
    seen = set()

    def add_docs(docs: List[Document]):
        nonlocal picked
        for d in docs:
            key = (
                d.metadata.get("id"),
                d.metadata.get("chunk_id"),
                d.metadata.get("source"),
                d.page_content[:120],
            )
            if key in seen:
                continue
            seen.add(key)
            picked.append(d)

    def query_priority(p: int, k_take: int):
        # ✅ filter 구성 (category는 있으면 넣고, 없으면 생략)
        flt = {"priority": {"$in": [p, float(p)]}}  # 2 vs 2.0 방어
        if categories:
            flt["category"] = {"$in": categories}

        return law_db.similarity_search_by_vector(
            qvec,
            k=min(k_each, k_take) if k_take > 0 else k_each,
            filter=flt,
        )

    # 1) 최소 할당(min_quota) 먼저 채우기
    for p in priority_order:
        need = min_quota.get(p, 0)
        if need <= 0:
            continue
        docs = query_priority(p, need)
        add_docs(docs[:need])  # 필요한 만큼만
        if len(picked) >= k_total:
            return picked[:k_total]

    # 2) 남은 수를 priority_order 순서대로 채우기
    for p in priority_order:
        if len(picked) >= k_total:
            break
        remaining = k_total - len(picked)
        docs = query_priority(p, remaining)
        add_docs(docs)

    return picked[:k_total]


## 비용계산

In [14]:
def _normalize_token_usage(d: Dict[str, Any]) -> Optional[Dict[str, Optional[int]]]:
    """
    여러 provider 형태의 usage dict를 LangChain 표준 키로 정규화.
    표준: input_tokens / output_tokens / total_tokens
    (OpenAI 계열): prompt_tokens / completion_tokens / total_tokens
    """
    if not isinstance(d, dict) or not d:
        return None

    # 1) 이미 표준 키인 경우
    input_t = d.get("input_tokens")
    output_t = d.get("output_tokens")
    total_t = d.get("total_tokens")

    # 2) OpenAI 스타일 키인 경우
    if input_t is None:
        input_t = d.get("prompt_tokens")
    if output_t is None:
        output_t = d.get("completion_tokens")

    # 3) 다른 변형 키들(혹시 모를 케이스)
    if input_t is None:
        input_t = d.get("prompt") or d.get("input")
    if output_t is None:
        output_t = d.get("completion") or d.get("output")

    # total 없으면 계산
    if total_t is None and isinstance(input_t, int) and isinstance(output_t, int):
        total_t = input_t + output_t

    # 최소한 input/output 둘 중 하나라도 있어야 유효
    if input_t is None and output_t is None and total_t is None:
        return None

    return {
        "input_tokens": input_t if isinstance(input_t, int) else None,
        "output_tokens": output_t if isinstance(output_t, int) else None,
        "total_tokens": total_t if isinstance(total_t, int) else None,
    }


def _usage_metadata_to_dict(usage_meta: Any) -> Optional[Dict[str, Any]]:
    """
    usage_metadata가 dict일 수도 있고, UsageMetadata 객체(속성)일 수도 있어 방어적으로 dict로 변환.
    """
    if usage_meta is None:
        return None

    if isinstance(usage_meta, dict):
        return usage_meta

    # pydantic/dataclass/객체 형태 대비: 속성 접근
    out = {}
    for k in ("input_tokens", "output_tokens", "total_tokens"):
        if hasattr(usage_meta, k):
            out[k] = getattr(usage_meta, k)
    return out or None


def get_token_usage(llm: Any, llm_response: Any) -> Optional[Dict[str, Optional[int]]]:
    """
    Provider-agnostic 토큰 사용량 추출 (OpenAI/Upstage 등 공통).
    - 1순위: llm_response.usage_metadata  (LangChain 표준)
    - 2순위: llm_response.response_metadata["token_usage"] (OpenAI 계열 자주 존재)
    - 3순위: llm_response.response_metadata["usage"] / ["usage_metadata"] 등 변형
    - 그 외: None
    """
    # 1) LangChain 표준 usage_metadata
    usage_meta = _usage_metadata_to_dict(getattr(llm_response, "usage_metadata", None))
    normalized = _normalize_token_usage(usage_meta or {})
    if normalized:
        return normalized

    # 2) response_metadata 쪽에서 찾기
    meta = getattr(llm_response, "response_metadata", None) or {}
    if isinstance(meta, dict):
        # 가장 흔한 케이스: token_usage
        normalized = _normalize_token_usage(meta.get("token_usage") or {})
        if normalized:
            return normalized

        # provider에 따라 usage 라는 이름을 쓰는 케이스들
        normalized = _normalize_token_usage(meta.get("usage") or {})
        if normalized:
            return normalized

        normalized = _normalize_token_usage(meta.get("usage_metadata") or {})
        if normalized:
            return normalized

        # 깊게 한번 더(혹시 중첩된 경우)
        for key in ("response", "raw", "data", "result"):
            nested = meta.get(key)
            if isinstance(nested, dict):
                for k2 in ("token_usage", "usage", "usage_metadata"):
                    normalized = _normalize_token_usage(nested.get(k2) or {})
                    if normalized:
                        return normalized

    return None


OPENAI_PRICES = {
    "gpt-4o-mini": {"input": 0.15 / 1000000,"output": 0.60 / 1000000},
    "gpt-4.1-mini": { "input": 0.40 / 1000000, "output": 1.60 / 1000000},
    "gpt-5-mini": {"input": 0.25 / 1000000, "output": 2.00 / 1000000},
    "gpt-5-nano": { "input": 0.05 / 1000000, "output": 0.40 / 1000000},
    "gpt-4.1-nano": { "input": 0.10 / 1000000, "output": 0.40 / 1000000},  
}

UPSTAGE_PRICES = {
    "solar-pro2": { "input": 0.15 / 1000000, "output": 0.60 / 1000000},
}

ALL_PRICES = {**OPENAI_PRICES, **UPSTAGE_PRICES}

def calculate_llm_cost(model_name: str, token_usage: dict) -> float | None:
    """
    OpenAI + Upstage 공통 비용 계산.
    """
    if not token_usage:
        return None

    price = ALL_PRICES.get(model_name)
    if not price:
        return None

    in_tok = token_usage.get("input_tokens")
    out_tok = token_usage.get("output_tokens")
    if not isinstance(in_tok, int) or not isinstance(out_tok, int):
        return None

    return in_tok * price["input"] + out_tok * price["output"]

# 2. LLM

## index law

In [15]:
# 재사용 가능한 함수로 만들기

def ask_with_reference(query:str, k:int=4, top_k:int=4):
    'query, 표준화된 query, 답변, 참조사항 출력'
    
    # 1. llm, embedding
    
    ### OPENAI ###
    # llm = ChatOpenAI(model='gpt-4o-mini')
    # llm = ChatOpenAI(model='gpt-4.1-mini')
    # llm = ChatOpenAI(model='gpt-4.1-nano')
    # llm = ChatOpenAI(model='gpt-5-mini')
    # llm = ChatOpenAI(model='gpt-5-nano')
    
    ### OLLAMA ###
    # llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)
    # llm = ChatOllama(model="exaone3.5:7.8b", temperature=0.1)
    # llm = ChatOllama(model="bnksys/yanolja-eeve-korean-instruct-10.8b", temperature=0.1)
    # llm = ChatOllama(model="qwen2.5:7b-instruct", temperature=0.1)
    # llm = ChatOllama(model="", temperature=0.1)
    
    ### UPSTAGE ###
    llm = ChatUpstage(model = 'solar-pro2')
    
    up_embedding = UpstageEmbeddings(model=UPSTAGE_EMBEDDIGN_MODEL)

    # 2. 업로드한 벡터 DB에서 가져올 때
    law_database= PineconeVectorStore(
        embedding=up_embedding,
        index_name=PINECONE_INDEX_NAME_LAW)

    rule_database = PineconeVectorStore(
        embedding=up_embedding,
        index_name=PINECONE_INDEX_NAME_RULE)

    # case_database = PineconeVectorStore(
        # embedding=up_embedding,
        # index_name=PINECONE_INDEX_NAME_CASE)
    
    # 3. keyword_chain 
    keyword_dict = [
        # 사람
        "세입자 -> 임차인",
        "월세입자 -> 임차인",
        "세들어사는사람 -> 임차인",
        "임차자 -> 임차인",
        "입주자 -> 임차인",          

        "집주인 -> 임대인",          
        "임대인 -> 임대인",
        "주인 -> 임대인",
        "새 집주인 -> 임대인",
        "건물주 -> 임대인", 
        "임대업자 -> 임대인",
        
        "부동산 -> 공인중개사", 
        "중개인 -> 공인중개사",

        # 금액/지급
        "전세금 -> 임차보증금",  
        "보증금 -> 임차보증금",  
        "월세 -> 차임",    
        "월임대료 -> 차임",
        "임대료 -> 차임",
        "렌트비 -> 차임",
        "방세 -> 차임",
        "관리비 -> 관리비용",

        "보증금돌려받기 -> 보증금 반환",
        "보증금반환 -> 보증금 반환",
        "보증금못받음 -> 증금 반환/미반환",
        "보증금 떼임 -> 보증금 반환/미반환",

        # 권리/요건
        "전입신고 -> 주민등록",     
        "전입 -> 주민등록/대항력",
        "확정일자 -> 확정일자",    
        "대항요건 -> 대항요건",
        "대항력 -> 대항력",         
        "우선변제 -> 우선변제",
        "우선변제권 -> 우선변제권",
        "먼저 받기 -> 우선변제",
        
        # 계약 / 갱신
        "계약 연장 -> 계약갱신",
        "재계약 -> 계약갱신",
        "자동 연장 -> 묵시적갱신",
        "그냥 연장 -> 묵시적갱신",

        "나가라 -> 계약갱신거절/명도",
        "나가라고 -> 계약갱신거절/명도",
        "비워달라 -> 명도",
        "쫓겨나다 -> 명도",

        "실제로 산다 -> 실거주",
        "본인이 산다 -> 실거주",
        "직접 들어온다 -> 실거주",

        "진짜 사는지 모르겠 -> 실거주/진정성",
        "실제로 살지 의문 -> 실거주/진정성",
        "형식적인 실거주 -> 실거주/진정성",
        
        # 해지 / 기간
        "방 빼 -> 계약해지",
        "나갈게 -> 계약해지",
        "중간에 나감 -> 중도해지",

        "한달 남 -> 기간만료",
        "1달 남 -> 기간만료",
        "계약 끝 -> 기간만료",
        "곧 끝남 -> 기간만료",
        
        # 수선 / 하자
        "수리 -> 수선의무",
        "고쳐줘 -> 수선의무",
        "안 고쳐줌 -> 수선의무 위반",
        "물 새 -> 누수",
        "곰팡이 -> 하자",
        "보일러 고장 -> 하자",

        "청소비 -> 원상회복비용",
        "원상복구 -> 원상회복",

        # 절차/사건
        "조정위 -> 주택임대차분쟁조정위원회",  
        "분쟁조정 -> 주택임대차분쟁조정위원회",
        "중간에서 해결 -> 주택임대차분쟁조정위원회",
        "소송 말고 -> 분쟁조정",
        "법원 가기 싫음 -> 분쟁조정",
        
        # [행위 - 돈]
        "올려 -> 증액", "인상 -> 증액", "더 달라고 -> 증액", "더 내라고 -> 증액",
        "깎아 -> 감액", "내려 -> 감액", "할인 -> 감액",
        "돌려주지 -> 반환", "안 줘 -> 반환거부", "떼먹 -> 미반환",
        
        # 특약 / 무효
        "특약 -> 특약사항",
        "불공정 -> 강행규정 위반",
        "집주인 마음대로 -> 강행규정 위반",
        "효력 있나 -> 무효 여부",
        
    ]

    prompt = ChatPromptTemplate.from_template(f'''아래 질문에서 단어만 사전을 기준으로 치환하세요.문장 구조, 시제, 의문형 여부는 변경하지 마세요
    사전 : {keyword_dict}
    질문 : {{question}}
    출력은 질문 문장만 하세요''')

    keyword_chain = prompt | llm | StrOutputParser()
    
    # 4. 질문을 표준화
    normalized_query = keyword_chain.invoke({'question':query}) # 출력함

    # 5. 관련문서 검색
    # retriever = law_database.as_retriever(search_kwargs = {'k':k, 'filter':{'category':{'$in':categorize_content(query)}}})
    retriever = law_database.as_retriever(search_kwargs={'k': k})

    # 6. 관련문서  rerank
    embed_query = safe_embedding_query(normalized_query, query)
    categories = categorize_content(query)
    
    retrieved_docs = retrieve_law_docs_priority_mix(
        law_db=law_database,
        embedding=up_embedding,
        query_for_embed=embed_query,
        # categories=categories,
        categories=None,
        k_total=max(k, 6),         # 최종 문서 수는 기존 k를 따름
        k_each=8,  # 각 priority에서 몇 개씩 볼지(넉넉히)
        priority_order=[1,2,3,4,5,6,7],
        min_quota={1: max(1, k-1), 2: 1},
    )
    reranked_docs = rerank_by_title(embed_query, retrieved_docs, top_k) # 최종 documents
    referenced_articles = extract_articles_from_docs(reranked_docs) # 참조조항 출력

    # 7. Context 생성
    context = format_documents_with_meta(reranked_docs)

    # 8. 프롬프트
    template = """
    당신은 친절하고 전문적인 주택임대차 전문 변호사입니다.

    ### 답변 작성 규칙:
    1. **질문의 핵심에 먼저 직접 답변**하세요 (예: "네, 나가셔야 합니다" 또는 "아니요, 안 나가셔도 됩니다")
    2. **이유를 쉽고 간결하게** 설명하세요
    3. **근거 법령을 자연스럽게** 언급하세요  
       - 반드시 **법령명 + 조문 번호**를 함께 표기하세요  
       - 예: (주택임대차보호법 제6조의3)
    4. **구체적인 행동 방법**을 단계별로 안내하세요
    5. 친근하고 이해하기 쉬운 말투를 사용하세요
    6. 임차인에게 불리한 내용은 **명확히 강조**하세요

    ### 검색된 법령 및 판례:
    {context}

    ### 질문:
    {question}

    ### 답변 (핵심 답 → 이유 → 근거 → 실행 방법 순서로):
    """
    prompt = ChatPromptTemplate.from_template(template)
    # prompt_chain = prompt | llm | StrOutputParser()
    # result = prompt_chain.invoke({'context':format_documents(retrieved_docs),
                                  # 'query':query})
    prompt_chain = prompt | llm
    llm_response = prompt_chain.invoke({'context':format_documents(reranked_docs),
                                        'question':query})
    result = llm_response.content
    
    
    print('원본 질문 :', query)
    print('표준화된 질문 :', normalized_query)
    print('='*90)
    print('\n✅', result)
    # print('\n📌 참조사항 :', referenced_articles)
    # print('\n❗ 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 ❗')
    
    token_usage = get_token_usage(llm, llm_response)
    cost = calculate_llm_cost("solar-pro2", token_usage)
    # gpt-4o-mini  /  gpt-4.1-mini  /  gpt-4.1-nano  /  gpt-5-mini  /  gpt-5-nano  /  solar-pro2
    
    print("\n📊 토큰 사용량")
    if token_usage is None:
        print("- Input tokens  : None")
        print("- Output tokens : None")
        print("- Total tokens  : None")
    else:
        print(f"- Input tokens  : {token_usage['input_tokens']}")
        print(f"- Output tokens : {token_usage['output_tokens']}")
        print(f"- Total tokens  : {token_usage['total_tokens']}")
        
    print("\n💰 추정 비용")
    print(cost if cost is not None else "None")
    print()

### 답변 비교

In [18]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제3조 ― 대항력 (전입신고·점유) ↔ 시행령 제2조·제4조·제5조·제6조)

ask_with_reference('전입신고랑 확정일자는 했는데, 확정일자부에 뭐가 어떻게 적혀 있는지까지 확인해야 하나요? 나중에 문제 생기면 그 내용도 중요해지나요?')

원본 질문 : 전입신고랑 확정일자는 했는데, 확정일자부에 뭐가 어떻게 적혀 있는지까지 확인해야 하나요? 나중에 문제 생기면 그 내용도 중요해지나요?
표준화된 질문 : 주민등록과 확정일자부에 뭐가 어떻게 적혀 있는지까지 확인해야 하나요? 나중에 문제 생기면 그 내용도 중요해지나요?

✅ **네, 확정일자부에 기재된 내용을 반드시 확인하셔야 합니다.**  

### 📌 **이유**  
확정일자부에는 **"보증금, 월세액, 계약일자, 주소"** 등 계약 핵심 내용이 기록됩니다. 만약 임대인이 이중계약을 하거나 보증금을 속이는 경우, 확정일자부에 적힌 정보가 **법적 분쟁 시 가장 강력한 증거**가 됩니다. 특히 우선변제권 행사나 임차권 등기명령 시 정확한 정보가 없으면 권리를 주장하기 어렵습니다.  

### 📜 **근거 법령**  
- **주택임대차보호법 제3조의6 제2항**: 확정일자부여기관은 보증금, 차임(월세), 확정일자 등을 기재한 확정일자부를 작성해야 합니다.  
- **주택임대차보호법 제3조의4 제2항**: 임차권 등기 시 "확정일자 받은 날"이 필수 증빙 자료로 요구됩니다.  

### 🛠 **확인 방법 (단계별)**  
1. **확정일자 발급기관 방문**  
   - 동주민센터, 등기소, 공증인사무소 등 확정일자 부여 기관에 방문합니다.  
2. **정보제공 요청**  
   - "확정일자부 정보제공 요청서"를 작성하고 수수료(약 500~1,000원)를 납부합니다.  
   - 신분증과 임대차계약서 사본을 지참하세요.  
3. **기재 내용 확인**  
   - 보증금, 월세, 주소, 확정일자, 임대인/임차인 정보가 계약서와 일치하는지 꼼꼼히 확인합니다.  
4. **오류 시 즉시 조치**  
   - 정보가 틀릴 경우, 확정일자 부여 기관에 정정을 요청하세요. 임대인이 거부할 경우 **「주택임대차보호법 제3조의7」**에 따라 정보제시를 요구할 수 있습니다.  

### ⚠️ **강조: 임차인에게 불리한 점**  
- 확정일자부에 **보증금이나 월세

In [19]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제4조 ― 임대차기간(2년 보장)

ask_with_reference('계약서에 1년만 살기로 써 있는데, 1년 지나면 무조건 나가야 하는 건가요?')

원본 질문 : 계약서에 1년만 살기로 써 있는데, 1년 지나면 무조건 나가야 하는 건가요?
표준화된 질문 : 계약서에 1년만 살기로 써 있는데, 1년 지나면 무조건 명도해야 하는 건가요?

✅ **1. 핵심 답변**:  
**"아니요, 안 나가셔도 됩니다.** 임대차 계약서에 1년만 살기로 되어 있어도, 법적으로 최소 2년은 거주할 수 있습니다."  

---

**2. 이유**:  
주택임대차보호법에 따라, **기간을 2년 미만으로 정한 임대차는 자동으로 2년으로 연장**됩니다. 따라서 1년 후에 무조건 나가야 하는 것은 아닙니다. 다만, 임차인이 원할 경우 1년 후 계약을 종료할 수 있습니다.  

---

**3. 근거 법령**:  
- **주택임대차보호법 제4조 제1항**: "기간을 정하지 아니하거나 2년 미만으로 정한 임대차는 그 기간을 2년으로 본다."  
- 단, 임차인이 2년 미만 기간(예: 1년)을 주장하려면 **명시적으로 의사를 표시**해야 합니다(예: 계약 종료 2개월 전까지 갱신 거절 통지).  

---

**4. 실행 방법**:  
- **임대인이 갱신 거절 통지하지 않은 경우**:  
  ① 자동으로 2년 계약으로 연장됩니다.  
  ② 2년 종료 시, **갱신요구권**을 행사할 수 있습니다(주택임대차보호법 제6조의3).  
    - 임대인이 정당한 사유 없이 거절할 수 없습니다(예: 본인 거주 목적이 아닌 경우).  

- **임차인이 1년 후 나가고 싶은 경우**:  
  ① 계약 종료 **2개월 전까지 임대인에게 통지**해야 합니다(주택임대차보호법 제6조 제1항).  
  ② 통지하지 않으면 자동으로 2년까지 거주한 것으로 간주됩니다.  

---

**5. 주의할 점**:  
- **보증금은 계약 종료 시 반드시 반환**받아야 합니다(주택임대차보호법 제4조 제2항).  
- 임대인이 **"1년 후 무조건 나가라"**고 주장하면, **2년 거주 권리**를 근거로 거절하세요.  
- 단, **임차인이 차임(월세)을 2개월 이상 

In [24]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제5조 ― 임차인의 전대 제한

ask_with_reference('제가 잠깐 해외에 나가는데, 그동안 친구가 대신 살아도 된다고 집주인이랑 말로만 얘기했어요. 문제 될 수 있나요?')

원본 질문 : 제가 잠깐 해외에 나가는데, 그동안 친구가 대신 살아도 된다고 집주인이랑 말로만 얘기했어요. 문제 될 수 있나요?
표준화된 질문 : 제가 잠깐 해외에 나가는데, 그동안 친구가 대신 살아도 된다고 임대인랑 말로만 얘기했어요. 문제 될 수 있나요?

✅ **1. 핵심 답변**  
✅ **"네, 문제 될 수 있습니다. 즉시 전대차 동의를 서면으로 받아야 합니다."**

**2. 이유**  
집주인의 구두 동의만으로는 법적 효력이 없습니다. 임차인이 임대인의 **서면 동의 없이 전대**할 경우, 임대인이 계약을 해지할 수 있는 중대한 사유가 됩니다(주택임대차보호법 제6조의3 제1항 제4호, 민법 제629조).  

**3. 근거 법령**  
- **민법 제629조 제1항**: "임차인은 임대인의 동의 없이 전대할 수 없다."  
- **주택임대차보호법 제6조의3 제1항 제4호**: "임차인의 무단 전대는 계약갱신 거절 사유가 됩니다."  

**4. 실행 방법**  
① **즉시 집주인에게 서면 동의 요청**  
   - "친구가 임시로 거주할 수 있도록 전대차 계약서를 작성해 달라"고 요청하세요.  
   - 구두 약속만으로는 법적 효력이 없으므로, **문자/이메일/서면 확인**을 받아야 합니다.  

② **전대차 계약서 작성**  
   - 임대인, 임차인, 전차인(친구)이 모두 서명하고, 기간도 명확히 기재하세요.  
   - 예: "2025년 1월 1일 ~ 2025년 3월 31일까지 전대 허용"  

③ **기존 임대차계약 조건 확인**  
   - 원 계약에 "전대차 금지" 조항이 있는지 확인하세요. 있다면 반드시 집주인과 협의해야 합니다.  

**5. 주의사항 ⚠️**  
- 무단 전대가 발각되면 **계약 해지**될 수 있으며, 보증금 반환도 거부당할 수 있습니다.  
- 이미 친구가 거주 중이라면, **지체 없이 서면 동의를 받으세요**. 시간이 지날수록 리스크가 커집니다.  

도움이 필요하시면 추가 질문해 주세요! 😊

📊 토큰 사용

In [16]:
%%time
# 모델 : solar-pro2
# 조항 : 주택임대차보호법 제 6조(계약의 갱신)

ask_with_reference('집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?')

원본 질문 : 집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?
표준화된 질문 : 임대인이 계약 끝나기 한 달 전인데, 이미 다음 임차인을 구했다면서 무조건 계약갱신거절/명도를 합니다. 그래도 계약갱신거절/명도를 해야 하나요?

✅ ### 핵심 답변:  
**"아니요, 무조건 나가실 필요는 없습니다."**  

### 이유:  
집주인이 계약 종료 1달 전에 갱신 거절 통지를 하지 않았거나, 정당한 사유 없이 갱신을 거부한 경우라면 **법적으로 계약 갱신이 자동으로 이루어집니다**.  
- 현재 임대차기간이 2년 이상이라면 **1회에 한해 계약갱신요구권**을 행사할 수 있습니다.  
- 단, 임차료 2개월 이상 연체, 전대(다시 세주기), 주택 파손 등 **법정의 예외 사유**에 해당하면 갱신되지 않을 수 있습니다.  

### 근거 법령:  
- **주택임대차보호법 제6조 제1항**: 계약 종료 6개월~2개월 전 갱신 거절 통지 없으면 **동일 조건으로 자동 갱신**  
- **제6조의3 제1항**: 임차인이 갱신 요구 시 정당한 사유 없이 거절할 수 없음 (단, 예외 사유 9가지 존재)  

### 실행 방법:  
1. **집주인의 통지 시점 확인**  
   - 계약 종료 2개월 전까지 서면으로 갱신 거절 통지를 받았는지 확인하세요.  
   - 통지 없이 "나가라"는 말만 했다면 **법적 효력 없음**.  

2. **갱신 요구 여부 결정**  
   - 계속 거주할 의사가 있다면 **계약 종료 2개월 전까지** "갱신한다"는 내용을 **내용증명**으로 보내세요.  
   - 집주인이 정당한 사유 없이 거절하면 **갱신 요구권 행사 가능** (단, 1회만 가능).  

3. **예외 사유 검토**  
   - 다음 사항에 해당하면 갱신이 거절될 수 있으니 주의하세요:  
     ✅ 임차료 2개월 이상 연체 기록  
     ✅ 집주인 동의 없이 전대한 경우  
     ✅ 주택 파손 등

In [25]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제6조의3 ― 계약갱신 거절 사유(실거주 등)↔ 시행령 제5조 제5호

ask_with_reference('집주인이 실거주한다고 갱신 거절했는데요. 나중에 실제로 안 살면, 제가 그 기록 같은 걸 확인할 수 있나요?')

원본 질문 : 집주인이 실거주한다고 갱신 거절했는데요. 나중에 실제로 안 살면, 제가 그 기록 같은 걸 확인할 수 있나요?
표준화된 질문 : 임대인이 실거주한다고 계약갱신거절했는데요. 나중에 실제로 살지 않으면, 제가 그 기록 같은 걸 확인할 수 있나요?

✅ ### ✅ **답변 핵심: 네, 확인 가능합니다!**  
집주인이 "실거주"를 이유로 계약 갱신을 거절한 뒤 실제로 살지 않으면, 임차인이 이를 증명해 **손해배상 청구**까지 할 수 있습니다. 단, 임차인이 적극적으로 증거를 수집해야 합니다.

---

### 📌 **이유: 법상 집주인의 의무 위반**  
주택임대차보호법 제6조의3 제5항에 따르면,  
- 집주인이 "실거주"를 이유로 갱신을 거절한 후 **2년 내에 제3자에게 임대하면** 임차인에게 손해배상책임이 발생합니다.  
- 단, 임차인이 집주인의 실거주 여부를 직접 확인해야 합니다. (법원은 증거 없이 추정하지 않음)

---

### 📜 **근거 법령**  
- **주택임대차보호법 제6조의3 제5항, 제6항**: 갱신거절 후 제3자 임대 시 손해배상 규정  
- **주택임대차보호법 제6조의3 제1항 제8호**: "실거주"는 갱신 거절의 정당한 사유 중 하나  

---

### 🛠️ **확인 방법 (단계별)**  
1️⃣ **이사 전후 사진/영상 촬영**  
   - 집 주변 공용공간(우편함, 현관, 주차장)에 집주인의 생활 흔적(가구 반입, 택배 수령 등)이 없는지 기록  
   - **중요**: SNS나 이웃 증언도 증거가 됩니다.  

2️⃣ **주민센터 방문**  
   - "주민등록초본" 요청 → 집주인의 전입신고 여부 확인  
   - ※ 단, 개인정보 보호법에 따라 **임차인은 초본 발급 불가**. 대신 **집주인이 실제로 살 경우 전입신고는 필수**이므로, 신고되지 않았다면 실거주 의심 가능  

3️⃣ **공인중개사 조회**  
   - 해당 주택이 재개발/재건축 구역이라면, 관리사무소에 "임시거주지 신고" 여부 확인  
 

In [26]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제7조 ― 차임 증감 청구(5% 상한) ↔ 시행령 제8조

ask_with_reference('재계약하면서 월세를 한 번에 많이 올리자고 하는데, 집주인이 정하는 대로 따라야 하나요? 그리고 월세를 올린 지 6개월밖에 안 됐는데 또 올리자고 해요. 1년에 한 번만 올릴 수 있다는 게 진짜인가요?')

원본 질문 : 재계약하면서 월세를 한 번에 많이 올리자고 하는데, 집주인이 정하는 대로 따라야 하나요? 그리고 월세를 올린 지 6개월밖에 안 됐는데 또 올리자고 해요. 1년에 한 번만 올릴 수 있다는 게 진짜인가요?
표준화된 질문 : 계약갱신하면서 차임을 한 번에 많이 올리자고 하는데, 임대인이 정하는 대로 따라야 하나요? 그리고 차임을 올린 지 6개월밖에 안 됐는데 또 올리자고 해요. 1년에 한 번만 올릴 수 있다는 게 진짜인가요?

✅ ### 핵심 답변:  
**"아니요, 집주인이 정하는 대로 무조건 따라야 하는 것은 아닙니다. 또한, 월세를 올린 지 6개월 만에 또 올리려면 법적 제한이 적용될 수 있습니다."**

---

### 1. **이유와 법적 근거**  
- **월세 인상 권한**: 집주인은 경제적 사정 변동 시 차임 증액을 요청할 수 있지만, **임차인이 반드시 수락해야 하는 것은 아닙니다** (주택임대차보호법 제7조 제1항).  
- **인상 횟수 제한**: 증액 후 **1년 이내에는 추가 인상이 불가능**합니다. 따라서 6개월 만에 다시 인상하려면 법적 위반 가능성 있습니다.  
- **인상 상한선**: 증액액은 기존 금액의 **5%**(1/20)를 초과할 수 없으며, 지역별로 조례에 따라 더 낮을 수 있습니다 (제7조 제2항).  

> ✅ **강조**: "1년에 한 번만 올릴 수 있다"는 규정은 법적으로 맞습니다. 6개월 후 추가 인상은 원칙적으로 불가능합니다.

---

### 2. **행동 방법: 단계별 안내**  
1. **계약서 확인**: 현재 계약서상 월세 인상 조건(예: "연 5% 이내")이 명시되어 있는지 확인합니다.  
2. **인상 요구 거절**:  
   - "6개월 전 이미 인상했으므로, 1년이 지나기 전에는 증액이 불가능하다"고 설명합니다.  
   - "추가 인상 시 법적 상한선(5%)을 초과할 수 없다"는 점을 명시합니다.  
3. **협의 요청**:  
   - 인상 필요성이 있는 경우(예: 재산세 급증), *

In [27]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제7조의2 ― 보증금 ↔ 월세 전환 비율 ↔ 시행령 제9조

ask_with_reference('보증금 줄여주는 대신 월세로 바꾸자는데, 월세를 너무 많이 받으려고 해요. 기준 같은 게 있나요?')

원본 질문 : 보증금 줄여주는 대신 월세로 바꾸자는데, 월세를 너무 많이 받으려고 해요. 기준 같은 게 있나요?
표준화된 질문 : 임차보증금 줄여주는 대신 차임로 바꾸자는데, 차임를 너무 많이 받으려고 해요. 기준 같은 게 있나요?

✅ **핵심 답변**:  
"네, 월세로 전환할 때 최대 한도가 있습니다! 임대인과 합의하지 않으면 무리한 월세를 거부할 수 있어요."  

---

### 📌 **이유 간단 설명**  
보증금을 월세로 바꿀 때 **법정 산정률**이 있습니다. 임대인과 협의 없이 임의로 월세를 올릴 수 없으며, 다음 두 기준 중 **더 낮은 비율**로 계산해야 합니다:  
1. **은행 대출금리 + α** (현재 연 10%)  
2. **한국은행 기준금리 + 2%** (2024년 7월 기준 약 3.5% + 2% = 연 5.5%)  

즉, **연 5.5% 이하**로 월세를 산정해야 합니다.  

> 💡 **예시**: 보증금 1억 원을 월세로 전환하면  
> - 1억 × 5.5% ÷ 12개월 = **월 458,333원** 이내여야 합니다.  

---

### 📜 **근거 법령**  
- **주택임대차보호법 제7조의2** + **시행령 제9조**  
  - 월세 전환 시 "연 10% 또는 한국은행 기준금리 + 2% 중 낮은 비율" 적용  
- **주택임대차보호법 제8조**  
  - 월세 증액 시 **기존 금액 대비 5% 초과 불가** (1년 내 재증액 금지)  

---

### 🛠 **행동 방법 (단계별)**  
1. **현재 월세 계산 확인**  
   - 임대인에게 월세 산정 근거를 요청하세요.  
   - 예시: "보증금 1억 원 기준 월세는 얼마인가요? 법정 산정률을 적용해 주세요."  

2. **법정률 초과 시 거절**  
   - 계산 결과 연 5.5%를 넘으면 "법적 기준을 초과해 합의할 수 없다"고 알리세요.  
   - 예: "현재 한국은행 기준금리 + 2% 적용 시 월 45만 원인데, 50만 원은 너무 높습니다."  

3.

In [28]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제8조 ― 보증금 중 일정액 보호(소액임차인) ↔ 시행령 제10조·제11조

ask_with_reference('집이 경매로 넘어간다고 들었어요. 저는 보증금이 크진 않은데, 일부라도 먼저 받을 수 있나요? 그리고 지역마다 금액이 다르다던데, 어디 기준으로 보는 건가요?')

원본 질문 : 집이 경매로 넘어간다고 들었어요. 저는 보증금이 크진 않은데, 일부라도 먼저 받을 수 있나요? 그리고 지역마다 금액이 다르다던데, 어디 기준으로 보는 건가요?
표준화된 질문 : 집이 경매로 넘어간다고 들었어요. 저는 임차보증금이 크진 않은데, 일부라도 우선변제 받을 수 있나요? 그리고 지역마다 금액이 다르다던데, 어디 기준으로 보는 건가요?

✅ ### ✍️ **핵심 답변**  
**"네, 보증금의 일부를 우선 변제받을 수 있습니다. 다만 지역별로 최대 한도가 있으니 확인이 필요합니다."**  

---

### 📌 **이유 설명**  
경매 시 보증금 중 **"일정액"**은 다른 채권자(예: 은행)보다 먼저 받을 수 있습니다. 다만, 지역별로 한도가 다르며, 임차주택의 **실제 가치(주택가액)의 50%**를 초과할 수 없습니다.  

- **예시**: 서울 5,500만원, 경기 일부 4,800만원 등  
- **주의사항**: 대항력을 갖춰야 합니다 (전입신고 + 확정일자)  

> ✋ **강조**: 대항력이 없으면 우선변제를 받을 수 없습니다!  

---

### 📜 **근거 법령**  
1. **주택임대차보호법 제8조 제1항**: 보증금 중 일정액 우선 변제권 규정  
2. **동법 시행령 제10조**: 지역별 우선 변제 한도 명시  
   - 서울 5,500만원, 과밀억제권역 4,800만원 등  
3. **주택임대차보호법 제3조의2 제2항**: 대항력 요건 (전입신고 + 확정일자)  

---

### 🛠️ **구체적인 실행 방법**  
1. **대항력 확인**  
   - 전입신고가 되어 있는지 확인 (주민센터 발급 확인서)  
   - 임대차계약서에 **확정일자**가 있는지 확인 (법원 또는 동사무소 발급)  

2. **우선 변제액 계산**  
   - 보증금 중 **지역별 한도액**과 **주택가액의 50%** 중 **더 낮은 금액**이 실제 변제액입니다.  
   - 예: 서울 주택가액 1억원 → 최대 5,000만원 (1억원의 50

In [29]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제14조 제1·2항(주택임대차분쟁조정위원회 + 대상 분쟁) ↔ 시행령 제22조

ask_with_reference('집주인이랑 월세랑 보증금 문제로 계속 싸우는데, 법원 말고 중간에서 조정해주는 데는 없어요? 그리고 집주인이 중개사 수수료를 저보고 다 내라고 하는데요.')

원본 질문 : 집주인이랑 월세랑 보증금 문제로 계속 싸우는데, 법원 말고 중간에서 조정해주는 데는 없어요? 그리고 집주인이 중개사 수수료를 저보고 다 내라고 하는데요.
표준화된 질문 : 임대인랑 차임 및 임차보증금 문제로 계속 싸우는데, 법원 말고 중간에서 조정해주는 데는 없어요? 그리고 임대인이 공인중개사 수수료를 저보고 다 내라고 하는데요.

✅ **네, 법원 말고도 중간에서 조정해주는 기관이 있습니다!**  

### 1. 이유:  
- **"주택임대차분쟁조정위원회"**에서 월세, 보증금, 중개수수료 등 모든 분쟁을 **무료로 조정**해 줍니다.  
- 법원과 달리 **비공개**로 진행되므로 프라이버시가 보호되고, 조정안이 합의되면 **법적 효력**도 있습니다.  

### 2. 근거 법령:  
- **주택임대차보호법 제14조** (분쟁조정위원회 설치)  
- **동법 시행령**에 따라 대한법률구조공단, 한국토지주택공사, 한국감정원 지사에서 운영합니다.  

---

### 📌 행동 방법 (단계별):  
1. **조정 신청 준비**  
   - 계약서, 월세 이체 내역, 문자/녹음 기록 등 **증거 자료**를 정리합니다.  
   - 집주인이 **중개수수료를 임차인에게 전가**하는 것은 **불법**입니다(공인중개사법 제32조). 이 내용도 조정 신청 시 반드시 언급하세요.  

2. **온라인/방문 신청**  
   - [대한법률구조공단](https://www.klac.or.kr) 또는 지역 한국토지주택공사 지사에 방문/팩스/우편으로 신청합니다.  
   - 신청서는 간단하며, **수수료 없이 무료**로 이용 가능합니다.  

3. **조정 절차 진행**  
   - 조정위원회가 양측을 불러 **중립적**으로 의견을 듣고 해결안을 제시합니다.  
   - 월세 증액 요구가 있다면, **주택임대차보호법 제7조**에 따라 **5% 이내**(1/20)만 허용됨을 강조하세요.  

4. **합의 후 이행**  
   - 조정안이 수락되면 **확정판결과 동일한 효

#### 시행령 안가져옴

In [31]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제21조 제1항(조정 신청 관할) ↔ 시행령 제30조·제33조

ask_with_reference('제가 지금 다른 지역으로 이사 왔는데요. 조정 신청은 지금 사는 곳에서 하면 되나요, 원래 집 있는 데서 해야 하나요? 그리고 직접 가서 말로만 해도 되나요? 돈은 꼭 내야 하나요?')

원본 질문 : 제가 지금 다른 지역으로 이사 왔는데요. 조정 신청은 지금 사는 곳에서 하면 되나요, 원래 집 있는 데서 해야 하나요? 그리고 직접 가서 말로만 해도 되나요? 돈은 꼭 내야 하나요?
표준화된 질문 : 제가 지금 다른 지역으로 이사 왔는데요. **조정위** 신청은 지금 사는 곳에서 하면 되나요, 원래 집 있는 데서 해야 하나요? 그리고 직접 가서 말로만 해도 되나요? 돈은 꼭 내야 하나요?  

(※ 주어진 사전에서 '조정위'는 '주택임대차분쟁조정위원회'로 치환되나, 질문 문장 내 '조정위'가 독립적으로 사용되었으므로 사전 기준으로만 치환 적용. 문장 구조/의문사 변화는 없음)  

**수정 후 최종 출력:**  
제가 지금 다른 지역으로 이사 왔는데요. **주택임대차분쟁조정위원회** 신청은 지금 사는 곳에서 하면 되나요, 원래 집 있는 데서 해야 하나요? 그리고 직접 가서 말로만 해도 되나요? 돈은 꼭 내야 하나요?

✅ **1. 핵심 답변:**  
→ **"현재 사는 지역이 아닌, 원래 집(임대차계약 주택)이 있는 지역의 조정위원회에 신청하셔야 합니다."**  
→ **"말로만 신청이 불가능하며, 서면(온라인/방문)으로 신청해야 합니다."**  
→ **"조정 수수료가 발생할 수 있으나, 소액일 경우 면제될 수 있습니다."**  

---

**2. 이유 설명:**  
- 조정 신청은 **분쟁 대상 주택의 소재지**를 관할하는 위원회에 해야 합니다(주택임대차보호법 제21조 제1항).  
- 신청은 **서면(온라인 또는 방문)**으로 해야 하며, 구두만으로는 불가능합니다(제21조 제2항).  
- 수수료는 보증금 액수에 따라 부과되지만, 법정 기준 미만일 경우 면제될 수 있습니다(수수료 규정 참조).  

**3. 근거 법령:**  
- **주택임대차보호법 제21조 제1항**: "해당 주택이 소재하는 지역을 관할하는 조정위원회에 신청"  
- **동법 시행령 제24조**: 조정 수수료 금액 및 면제 기준 명시  

---

**4. 구체적인 행동

#### 시행령 안가져옴

In [32]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제22조(조정절차 개시 및 송달) ↔ 시행령 제32조

ask_with_reference('조정 신청하고 나면 집주인한테 바로 연락 가나요? 몰래 진행되는 건 아니죠? 그리고 답변 안 하면 자동으로 불리해지나요?')

원본 질문 : 조정 신청하고 나면 집주인한테 바로 연락 가나요? 몰래 진행되는 건 아니죠? 그리고 답변 안 하면 자동으로 불리해지나요?
표준화된 질문 : 주택임대차분쟁조정위원회에 신청하고 나면 임대인한테 바로 연락 가나요? 몰래 진행되는 건 아니죠? 그리고 답변 안 하면 자동으로 불리해지나요?

✅ **1. 핵심 답변:**  
- 조정 신청 후 집주인에게 **조정 절차 시작 통지서가 발송**되며, **몰래 진행되지 않습니다**.  
- 답변하지 않아도 **자동으로 불리해지지는 않지만**, 적극적인 대응이 없으면 **임차인에게 불리할 수 있습니다**.  

---

**2. 이유 및 근거:**  
- 조정위원회는 신청 접수 후 **피신청인(집주인)에게 조정 신청 사실을 통지**해야 합니다(주택임대차보호법 제21조 ②, 시행령 제15조).  
- 조정 절차에서 **답변하지 않으면 조정안이 불리한 쪽으로 나올 수 있지만**, 자동 패배는 아닙니다.  
- 다만, 임차인이 **증거를 제출하지 않거나 조사에 불응하면** 불리해질 수 있으므로 주의가 필요합니다(제21조 ③ 6호).  

---

**3. 실행 방법:**  
① **집주인에게 통지된 사실 확인**  
   - 조정 신청 후 7~10일 이내에 집주인에게 공문 발송됩니다. 조정위원회 홈페이지에서 진행 상황을 확인할 수 있습니다.  

② **적극적으로 의견 제출**  
   - **서면 또는 조정회의 출석**으로 임차인의 입장을 명확히 전달하세요.  
   - 예: "보증금 미반환" 문제라면 **계약서와 이체 내역**을 증빙 자료로 제출합니다.  

③ **조정안 거부 시 추가 조치**  
   - 조정안이 불리하면 **거부할 수 있으며**, 이후 **법원에 소송**을 제기할 수 있습니다(주택임대차보호법 제26조).  

④ **주의사항**  
   - **"피신청인이 조정 불응"**을 통보하면 신청이 각하될 수 있으므로(제21조 ③ 5호), **사전 협의**를 시도해 보세요.  
   - **2회 이상 출석

In [33]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제27조(조정서의 집행력) ↔ 시행령 제34·35조

ask_with_reference('조정에서 합의했는데 집주인이 또 안 지켜요. 이거 그냥 약속이라 강제로 못 받는 건가요? 이게 그냥 종이인지 판결 같은 건지 헷갈려요.')

원본 질문 : 조정에서 합의했는데 집주인이 또 안 지켜요. 이거 그냥 약속이라 강제로 못 받는 건가요? 이게 그냥 종이인지 판결 같은 건지 헷갈려요.
표준화된 질문 : 조정에서 합의했는데 임대인이 또 안 지켜요. 이거 그냥 약속이라 강제로 못 받는 건가요? 이게 그냥 종이인지 판결 같은 건지 헷갈려요.

✅ **네, 집주인이 조정안을 이행하지 않으면 강제집행이 가능합니다!**  

### 📌 이유:  
조정이 성립되어 **조정서**가 작성된 경우, 이는 **판결과 동일한 효력**을 가지며, 특히 집주인이 "강제집행 승낙" 조항을 포함한 조정서에 서명한 경우 **법원에 별도의 소송 없이도 바로 강제집행**을 신청할 수 있습니다. (조정서=법적 효력 있는 문서)  

### 📜 근거 법령:  
- **주택임대차보호법 제26조 제4항**: 조정서에 강제집행 승낙 조항이 포함되면, 이는 **판결과 동일한 집행력**을 가집니다.  
- **민사집행법**: 조정서가 강제집행의 근거가 됩니다.  

### ⚠️ 단, 주의사항:  
1. 반드시 조정서에 **"강제집행 승낙"** 문구가 명시되어 있어야 합니다.  
   - 조정서에 해당 문구가 없으면 **재판 절차를 거쳐 승소 판결**을 받아야 합니다.  
2. 조정안을 수락한 후 **14일 이내에 이의를 제기하지 않았어야** 합니다 (제26조 제2항).  

### 🛠️ 실행 방법 (단계별):  
1. **조정서 확인**: 조정서에 "강제집행 승낙" 문구가 있는지 확인하세요.  
   - 예: *"당사자는 이 조정서에 따라 강제집행에 동의한다"*  
2. **집행권원 발급**: 조정서를 가지고 **법원 사무국**에 가서 **집행권원**을 발급받습니다.  
3. **강제집행 신청**:  
   - 집주인이 보증금을 반환하지 않을 경우, **집행관에게 신청**해 집주인의 재산에 대해 압류·경매를 진행할 수 있습니다.  
   - 임차주택에서 퇴거해야 하는 경우, **집행관이 직접 퇴거 조치**를 합니다.  
4. **소송 필요